In [1]:
import urllib.request
import tempfile

In [2]:
import pandas as pd
import sqlite3

In [3]:
# Temporary directory
temp_dir = tempfile.gettempdir()

In [4]:
# S3 URL for the SQLite file
s3_url = 'https://awsbucket3323.s3.amazonaws.com/capstone/raw-data/database.sqlite'

# Choose destination filepath I chose my desktop
temp_file_path = temp_dir + 'database.sqlite'

# Download the file
urllib.request.urlretrieve(s3_url, temp_file_path)

('C:\\Users\\SHAKO\\AppData\\Local\\Tempdatabase.sqlite',
 <http.client.HTTPMessage at 0x1fb67883b20>)

In [5]:
# Connect to the SQLite database
conn = sqlite3.connect(temp_file_path)

In [6]:
# locating the table names within are sqlite db
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)

[('sqlite_sequence',), ('Player_Attributes',), ('Player',), ('Match',), ('League',), ('Country',), ('Team',), ('Team_Attributes',)]


In [7]:
# Showing contents of League table
query = "SELECT * FROM League"
df = pd.read_sql_query(query, conn)
print(df.head())

      id  country_id                    name
0      1           1  Belgium Jupiler League
1   1729        1729  England Premier League
2   4769        4769          France Ligue 1
3   7809        7809   Germany 1. Bundesliga
4  10257       10257           Italy Serie A


In [8]:
# Saving League data as CSV file locally
# Manually uploading file to my AWS S3 Bucket using the S3 Management Console
df.to_csv(temp_dir + 'League.csv', index=False) 

In [9]:
# Showing contents of Country table
query = "SELECT * FROM Country"
df = pd.read_sql_query(query, conn)
print(df.head())

      id     name
0      1  Belgium
1   1729  England
2   4769   France
3   7809  Germany
4  10257    Italy


In [10]:
# Saving Country data as CSV file locally
# Manually uploading file to my AWS S3 Bucket using the S3 Management Console
df.to_csv(temp_dir + 'Country.csv', index=False) 

In [7]:
# Showing contents of Team table
query = "SELECT * FROM Team"
df = pd.read_sql_query(query, conn)
print(df.head())

   id  team_api_id  team_fifa_api_id     team_long_name team_short_name
0   1         9987             673.0           KRC Genk             GEN
1   2         9993             675.0       Beerschot AC             BAC
2   3        10000           15005.0   SV Zulte-Waregem             ZUL
3   4         9994            2007.0   Sporting Lokeren             LOK
4   5         9984            1750.0  KSV Cercle Brugge             CEB


In [8]:
# Saving Team data as CSV file locally
# Manually uploading file to my AWS S3 Bucket using the S3 Management Console
df.to_csv(temp_dir + 'Team.csv', index=False) 

In [12]:
# Showing contents of Player_Attributes table
query = "SELECT * FROM Player_Attributes"
df = pd.read_sql_query(query, conn)
print(df.head())

   id  player_fifa_api_id  player_api_id                 date  overall_rating  \
0   1              218353         505942  2016-02-18 00:00:00            67.0   
1   2              218353         505942  2015-11-19 00:00:00            67.0   
2   3              218353         505942  2015-09-21 00:00:00            62.0   
3   4              218353         505942  2015-03-20 00:00:00            61.0   
4   5              218353         505942  2007-02-22 00:00:00            61.0   

   potential preferred_foot attacking_work_rate defensive_work_rate  crossing  \
0       71.0          right              medium              medium      49.0   
1       71.0          right              medium              medium      49.0   
2       66.0          right              medium              medium      49.0   
3       65.0          right              medium              medium      48.0   
4       65.0          right              medium              medium      48.0   

   ...  vision  penalties 

In [13]:
# Converting 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])
print(df.head())

   id  player_fifa_api_id  player_api_id       date  overall_rating  \
0   1              218353         505942 2016-02-18            67.0   
1   2              218353         505942 2015-11-19            67.0   
2   3              218353         505942 2015-09-21            62.0   
3   4              218353         505942 2015-03-20            61.0   
4   5              218353         505942 2007-02-22            61.0   

   potential preferred_foot attacking_work_rate defensive_work_rate  crossing  \
0       71.0          right              medium              medium      49.0   
1       71.0          right              medium              medium      49.0   
2       66.0          right              medium              medium      49.0   
3       65.0          right              medium              medium      48.0   
4       65.0          right              medium              medium      48.0   

   ...  vision  penalties  marking  standing_tackle  sliding_tackle  \
0  ...    54.0 

In [14]:
# Saving Player Attributes as CSV file locally
# Manually uploading file to my AWS S3 Bucket using the S3 Management Console
df.to_csv(temp_dir + 'Player_Attributes.csv', index=False) 

In [7]:
query = "SELECT * FROM Match"
df = pd.read_sql_query(query, conn)
print(df.head())

   id  country_id  league_id     season  stage                 date  \
0   1           1          1  2008/2009      1  2008-08-17 00:00:00   
1   2           1          1  2008/2009      1  2008-08-16 00:00:00   
2   3           1          1  2008/2009      1  2008-08-16 00:00:00   
3   4           1          1  2008/2009      1  2008-08-17 00:00:00   
4   5           1          1  2008/2009      1  2008-08-16 00:00:00   

   match_api_id  home_team_api_id  away_team_api_id  home_team_goal  ...  \
0        492473              9987              9993               1  ...   
1        492474             10000              9994               0  ...   
2        492475              9984              8635               0  ...   
3        492476              9991              9998               5  ...   
4        492477              7947              9985               1  ...   

    SJA   VCH   VCD   VCA   GBH   GBD   GBA   BSH   BSD   BSA  
0  4.00  1.65  3.40  4.50  1.78  3.25  4.00  1.73  3

In [8]:
# Converting 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

In [9]:
# Adding 3 new columns for year, month, day
df['date_year'] = df['date'].dt.year
df['date_month'] = df['date'].dt.month
df['date_day'] = df['date'].dt.day

In [10]:
# Checking columns match with date column
print(df.head())

   id  country_id  league_id     season  stage       date  match_api_id  \
0   1           1          1  2008/2009      1 2008-08-17        492473   
1   2           1          1  2008/2009      1 2008-08-16        492474   
2   3           1          1  2008/2009      1 2008-08-16        492475   
3   4           1          1  2008/2009      1 2008-08-17        492476   
4   5           1          1  2008/2009      1 2008-08-16        492477   

   home_team_api_id  away_team_api_id  home_team_goal  ...   VCA   GBH   GBD  \
0              9987              9993               1  ...  4.50  1.78  3.25   
1             10000              9994               0  ...  3.25  1.85  3.25   
2              9984              8635               0  ...  2.65  2.50  3.20   
3              9991              9998               5  ...  6.50  1.50  3.75   
4              7947              9985               1  ...  1.65  4.50  3.50   

    GBA   BSH   BSD   BSA  date_year  date_month  date_day  
0  4.00

In [11]:
# Saving Match data as CSV file locally
# Manually uploading file to my AWS S3 Bucket using the S3 Management Console
df.to_csv(temp_dir + 'Match.csv', index=False) 

In [15]:
query = "SELECT * FROM Team_Attributes"
df = pd.read_sql_query(query, conn)
print(df.head())

   id  team_fifa_api_id  team_api_id                 date  buildUpPlaySpeed  \
0   1               434         9930  2010-02-22 00:00:00                60   
1   2               434         9930  2014-09-19 00:00:00                52   
2   3               434         9930  2015-09-10 00:00:00                47   
3   4                77         8485  2010-02-22 00:00:00                70   
4   5                77         8485  2011-02-22 00:00:00                47   

  buildUpPlaySpeedClass  buildUpPlayDribbling buildUpPlayDribblingClass  \
0              Balanced                   NaN                    Little   
1              Balanced                  48.0                    Normal   
2              Balanced                  41.0                    Normal   
3                  Fast                   NaN                    Little   
4              Balanced                   NaN                    Little   

   buildUpPlayPassing buildUpPlayPassingClass  ... chanceCreationShooting 

In [16]:
# Converting 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])
print(df.head())

   id  team_fifa_api_id  team_api_id       date  buildUpPlaySpeed  \
0   1               434         9930 2010-02-22                60   
1   2               434         9930 2014-09-19                52   
2   3               434         9930 2015-09-10                47   
3   4                77         8485 2010-02-22                70   
4   5                77         8485 2011-02-22                47   

  buildUpPlaySpeedClass  buildUpPlayDribbling buildUpPlayDribblingClass  \
0              Balanced                   NaN                    Little   
1              Balanced                  48.0                    Normal   
2              Balanced                  41.0                    Normal   
3                  Fast                   NaN                    Little   
4              Balanced                   NaN                    Little   

   buildUpPlayPassing buildUpPlayPassingClass  ... chanceCreationShooting  \
0                  50                   Mixed  ...       

In [17]:
# Saving Team_Attributes as CSV file locally
# Manually uploading file to my AWS S3 Bucket using the S3 Management Console
df.to_csv(temp_dir + 'Team_Attributes.csv', index=False) 

In [28]:
query = "SELECT * FROM Player"
df = pd.read_sql_query(query, conn)
print(df.head())

   id  player_api_id         player_name  player_fifa_api_id  \
0   1         505942  Aaron Appindangoye              218353   
1   2         155782     Aaron Cresswell              189615   
2   3         162549         Aaron Doran              186170   
3   4          30572       Aaron Galindo              140161   
4   5          23780        Aaron Hughes               17725   

              birthday  height  weight  
0  1992-02-29 00:00:00  182.88     187  
1  1989-12-15 00:00:00  170.18     146  
2  1991-05-13 00:00:00  170.18     163  
3  1982-05-08 00:00:00  182.88     198  
4  1979-11-08 00:00:00  182.88     154  


In [29]:
# Saving Player data as CSV file locally
# Manually uploading file to my AWS S3 Bucket using the S3 Management Console
df.to_csv(temp_dir + 'Player.csv', index=False) 

In [7]:
# Loop through the tables to see names, columns and rows
for table in tables:
    table_name = table[0]
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    print(f"Table: {table_name}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Number of Rows: {len(df)}\n")

Table: sqlite_sequence
Columns: ['name', 'seq']
Number of Rows: 7

Table: Player_Attributes
Columns: ['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating', 'potential', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression', 'interceptions', 'positioning', 'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes']
Number of Rows: 183978

Table: Player
Columns: ['id', 'player_api_id', 'player_name', 'player_fifa_api_id', 'birthday', 'height', 'weight']
Number of Rows: 11060

Table: Match
Columns: ['id', 'country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id'

## Combining the League and Country tables into one table

In [ ]:
# This was an attempt for the star schema but ended up not using Euro_League table
# Showcasing how I combined the tables and renamed the columns

In [9]:
# Creating the new combined table
create_query = '''
    CREATE TABLE IF NOT EXISTS Euro_Leagues (
        id INTEGER PRIMARY KEY,
        league_id INTEGER,
        country_id INTEGER,
        league_name TEXT,
        country_name TEXT
    );
'''

In [10]:
# Execute the new table 
cursor.execute(create_query)

In [11]:
# Insert data from League and Country tables
cursor.execute("SELECT id, country_id, name FROM League")
league_data = cursor.fetchall()

for row in league_data:
    cursor.execute("INSERT INTO Euro_Leagues (league_id, country_id, league_name, country_name) VALUES (?, ?, ?, ?)", (row[0], row[1], row[2], None))

cursor.execute("SELECT id, name FROM Country")
country_data = cursor.fetchall()

for row in country_data:
    cursor.execute("UPDATE Euro_Leagues SET country_name=? WHERE country_id=?", (row[1], row[0]))


In [12]:
# Checking columns and rows
# We can see there are the matching 11 rows with correct country names shown by the league name info
query = "SELECT * FROM Euro_Leagues"
df = pd.read_sql_query(query, conn)
print(df.head(20))

    id  league_id  country_id               league_name country_name
0    1          1           1    Belgium Jupiler League      Belgium
1    2       1729        1729    England Premier League      England
2    3       4769        4769            France Ligue 1       France
3    4       7809        7809     Germany 1. Bundesliga      Germany
4    5      10257       10257             Italy Serie A        Italy
5    6      13274       13274    Netherlands Eredivisie  Netherlands
6    7      15722       15722        Poland Ekstraklasa       Poland
7    8      17642       17642  Portugal Liga ZON Sagres     Portugal
8    9      19694       19694   Scotland Premier League     Scotland
9   10      21518       21518           Spain LIGA BBVA        Spain
10  11      24558       24558  Switzerland Super League  Switzerland


In [13]:
# Saving Euro_Leagues data as CSV file locally
# Manually uploading file to my AWS S3 Bucket using the S3 Management Console
df.to_csv(temp_dir + 'Euro_Leagues.csv', index=False) 

In [18]:
# Close the connection 
conn.close()